In [ ]:
import pandas as pd
import numpy as np
import csv
from google.colab import drive

# 1. Google Driveをマウント
drive.mount('/content/drive')

# 2. パスの設定（スペースが含まれる場合はそのまま引用符で囲みます）
# 'Colab Notebooks' などのフォルダ名もそのまま記述して問題ありません
input_path = 'your_path/your.csv'
output_path = 'your_path/your_result.csv'

print(f"✅ 設定完了。入力パス: {input_path}")


Mounted at /content/drive
✅ 設定完了。入力パス: /content/drive/MyDrive/Colab Notebooks/DATA-eijyu/FEH_00250011_260204111702.csv


In [ ]:
# ---------------------------------------------------------
# Step 1: Extract (データの抽出)
# ---------------------------------------------------------


with open(input_path, 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    all_rows = [row for row in reader]

# 地域名とコードの取得
raw_names = all_rows[10][13:]
# 属性ヘッダーの取得
attr_headers = [h.strip() if h.strip() != "" else f"col_{i}" for i, h in enumerate(all_rows[10][:12])]

# ---------------------------------------------------------
# Step 2: Transform (変換ロジック - 縦持ち形式)
# ---------------------------------------------------------
def transform_to_long_format(all_rows, attr_headers, raw_names):
    # データ本体の作成
    data_body = all_rows[11:]
    df_raw = pd.DataFrame(data_body)

    # 属性列と地域列を結合
    attr_part = df_raw.iloc[:, :12]
    value_part = df_raw.iloc[:, 13:13+len(raw_names)]
    df_combined = pd.concat([attr_part, value_part], axis=1)
    df_combined.columns = attr_headers + raw_names

    # 数値クレンジング
    def clean_num(val):
        if not val: return 0
        s = str(val).replace(',', '').strip()
        if s in ['', '-', '***']: return 0
        try: return int(float(s))
        except: return 0

    # 1. Melt処理 (地域名を縦持ちに変換)
    # ここでは pivot_table を行わず、このまま進めます
    df_long = df_combined.melt(
        id_vars=attr_headers,
        value_vars=raw_names,
        var_name='地域名',
        value_name='件数'
    )
    df_long['件数'] = df_long['件数'].apply(clean_num)

    # 2. 列名の特定と整理
    visa_col = attr_headers[8]    # 在留資格審査
    time_col = attr_headers[5]    # 時間軸（月次）
    status_col = attr_headers[11] # 在留資格審査の受理・処理

    # 3. 出入国管理局の抽出
    def get_bureau_prefix(name):
        for target in ["出入国在留管理局", "出入国管理局"]:
            if target in name:
                return name.split(target)[0]
        return ""

    df_long['出入国管理局'] = df_long['地域名'].apply(get_bureau_prefix)

    # 4. 指定の列名にリネーム
    df_long = df_long.rename(columns={
        visa_col: '在留資格審査',
        time_col: '时间轴（月次）',
        status_col: '在留資格審査の受理・処理'
    })

    # 5. 不要な列を削り、順番を整理
    # 必要な列だけをピックアップ
    final_cols = [
        '在留資格審査',
        '地域名',
        '出入国管理局',
        '时间轴（月次）',
        '在留資格審査の受理・処理',
        '件数'
    ]

    return df_long[final_cols]

# 実行
df_dashboard = transform_to_long_format(all_rows, attr_headers, raw_names)

# ---------------------------------------------------------
# Step 3: Load (書き出し)
# ---------------------------------------------------------
df_dashboard.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✅ 縦持ち形式での処理が完了しました：{output_path}")

✅ 縦持ち形式での処理が完了しました：/content/drive/MyDrive/Colab Notebooks/DATA-eijyu/processed_data.csv
